In [13]:
import tensorflow as tf
import keras
from keras.layers import Dense, Flatten
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

tf.config.run_functions_eagerly(True)

In [ ]:
print(tf.__version__)

### Load model

In [15]:
model = keras.models.load_model('model_ResNet50.h5')
model.compile(run_eagerly=True)

In [19]:
%%time
categories_dict = {"car":0, "light commercial vehicle":1,"truck":2,"minibus & van":3,"ambulance":4,"bus":5,"FP":6}
nb_classes = 7
df = pd.read_csv("SP01-CAM-04.csv", sep=';')
X = []
Y = []
vehicle_smart_pole = []
vehicle_day = []

# Get images in arrays
for index,row in enumerate(df.iterrows()):
    image_name = row[1]['path']
    image= tf.keras.preprocessing.image.load_img(image_name, color_mode='rgb', target_size=(224,224))
    image= np.array(image)
    X.append(image)
    vehicle_smart_pole.append(row[1]["smart_pole"])
    vehicle_day.append(row[1]["day"])

In [ ]:
%%time
X1 = np.array(X[:31667])
predictions1 = model.predict(X1)

In [16]:
block_size = 1000
initial_position = 31667
lenght = len(X)
continue_inference = True
while continue_inference:
    print("Initial position :", initial_position)
    if lenght > initial_position + block_size:
        current_x = np.array(X[initial_position:initial_position+block_size])
    else : 
        current_x = np.array(X[initial_position:])
        continue_inference = False
       
    initial_position += block_size
    current_predictions = model.predict(current_x)
    
    
    with open("inférence_save_SP01-CAM-03_part1.csv","a") as file:
        for pred in current_predictions:
            file.write(str(np.argmax(pred))+'\n')
            
    %reset_selective -f "^current_x$" 
    %reset_selective -f "^current_predictions$"

Initial position : 31667


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:
X1 = []
X1 = np.array(X[31667:])

In [ ]:
print(len(predictions1))

In [ ]:
with open("inférence_save_SP01-CAM-03_part1.csv","w") as file:
    file.write('category\n')
    for pred in predictions1:
        file.write(str(np.argmax(pred))+'\n')

In [ ]:
%%time
predictions2 = model.predict(X1)

In [ ]:
# to do : predictions = predictions1 + predictions2

In [ ]:
df['category'] = np.argmax(predictions1, axis=1)

In [ ]:
for index,row in enumerate(df.iterrows()):
    if 'bicycle' in row[1]['path']:
        smaller_df.loc[index,'category'] = 7
    elif 'motorcycle' in row[1]['path']:
        smaller_df.loc[index,'category'] = 8
        

In [ ]:
time_slots = []
for current_time in df['time']:
    current_hour = int(current_time.split('-')[0])
    if current_hour == 1: 
        current_hour = int(current_time.split('-')[1])
    if current_hour == 7 or current_hour == 8:
        time_slots.append("7h-9h")
    elif current_hour == 9 or current_hour == 10:
        time_slots.append("9h-11h")
    elif current_hour == 15 or current_hour == 16:
        time_slots.append("15h-17h")
    else:
        print("pb here :", current_time)

In [ ]:
df['time_slot'] = time_slots

In [ ]:
df.to_csv("inference_results_SP01-CAM-03_part1.csv",index=False)

# Add time slot

In [ ]:
df = pd.read_csv("SP01-CAM-04_inference.csv", sep=';')
for index,row in enumerate(df.iterrows()):
    if 'bicycle' in row[1]['path']:
        df.loc[index,'category'] = 7
    elif 'motorcycle' in row[1]['path']:
        df.loc[index,'category'] = 8

time_slots = []
for current_time in df['time']:
    current_hour = int(current_time.split('-')[0])
    if current_hour == 1: 
        current_hour = int(current_time.split('-')[1])
    if current_hour == 7 or current_hour == 8:
        time_slots.append("7h-9h")
    elif current_hour == 9 or current_hour == 10:
        time_slots.append("9h-11h")
    elif current_hour == 15 or current_hour == 16:
        time_slots.append("15h-17h")
    else:
        print("pb here :", current_time)

df['time_slot'] = time_slots
df.to_csv("SP01-CAM-04_results.csv",index=False)

In [ ]:
print(df)

# Inference for SP01-CAM-03_part2

In [ ]:
%%time
categories_dict = {"car":0, "light commercial vehicle":1,"truck":2,"minibus & van":3,"ambulance":4,"bus":5,"FP":6}
nb_classes = 7
df = pd.read_csv("database_for_inference_SP01-CAM-03_part2.csv")
X = []
Y = []
vehicle_smart_pole = []
vehicle_day = []
# Get images in arrays

#smaller_df = df.sample(n = 10_000, ignore_index=True)


for index,row in enumerate(df.iterrows()):
    image_name = row[1]['path']
    image=tf.keras.preprocessing.image.load_img(image_name, color_mode='rgb', target_size=(224,224))
    image=np.array(image)
    X.append(image)
    vehicle_smart_pole.append(row[1]["smart_pole"])
    vehicle_day.append(row[1]["day"])
    
X = np.array(X)
print("Len(X) :", len(X))

    
%%time
predictions = model.predict(X)

df['category'] = np.argmax(predictions, axis=1)
for index,row in enumerate(df.iterrows()):
    if 'bicycle' in row[1]['path']:
        smaller_df.loc[index,'category'] = 7
    elif 'motorcycle' in row[1]['path']:
        smaller_df.loc[index,'category'] = 8
        
time_slots = []
for current_time in df['time']:
    current_hour = int(current_time.split('-')[0])
    if current_hour == 1: 
        current_hour = int(current_time.split('-')[1])
    if current_hour == 7 or current_hour == 8:
        time_slots.append("7h-9h")
    elif current_hour == 9 or current_hour == 10:
        time_slots.append("9h-11h")
    elif current_hour == 15 or current_hour == 16:
        time_slots.append("9h-11h")
    else:
        print("pb here :", current_time)
        
df['time_slot'] = time_slots

df.to_csv("inference_results_SP01-CAM-03_part2.csv",index=False)

# Inference for SP01-CAM-04

In [ ]:
%%time
categories_dict = {"car":0, "light commercial vehicle":1,"truck":2,"minibus & van":3,"ambulance":4,"bus":5,"FP":6}
nb_classes = 7
df = pd.read_csv("database_for_inference_SP01-CAM-04.csv")
X = []
Y = []
vehicle_smart_pole = []
vehicle_day = []
# Get images in arrays

#smaller_df = df.sample(n = 10_000, ignore_index=True)


for index,row in enumerate(df.iterrows()):
    image_name = row[1]['path']
    image=tf.keras.preprocessing.image.load_img(image_name, color_mode='rgb', target_size=(224,224))
    image=np.array(image)
    X.append(image)
    vehicle_smart_pole.append(row[1]["smart_pole"])
    vehicle_day.append(row[1]["day"])
    
X = np.array(X)
print("Len(X) :", len(X))

    
%%time
predictions = model.predict(X)

df['category'] = np.argmax(predictions, axis=1)
for index,row in enumerate(df.iterrows()):
    if 'bicycle' in row[1]['path']:
        smaller_df.loc[index,'category'] = 7
    elif 'motorcycle' in row[1]['path']:
        smaller_df.loc[index,'category'] = 8
        
time_slots = []
for current_time in df['time']:
    current_hour = int(current_time.split('-')[0])
    if current_hour == 1: 
        current_hour = int(current_time.split('-')[1])
    if current_hour == 7 or current_hour == 8:
        time_slots.append("7h-9h")
    elif current_hour == 9 or current_hour == 10:
        time_slots.append("9h-11h")
    elif current_hour == 15 or current_hour == 16:
        time_slots.append("9h-11h")
    else:
        print("pb here :", current_time)
        
df['time_slot'] = time_slots

df.to_csv("inference_results_SP01-CAM-04.csv",index=False)

# From frame to majoritary vehicle 

In [ ]:
new_df = pd.DataFrame(columns=smaller_df.columns)

In [ ]:
for vehicle_id in set(smaller_df['id']):
    current_vehicle_df = smaller_df[smaller_df['id'] == vehicle_id]
    print(current_vehicle_df['id'].iloc[0])
    new_df = new_df.append(pd.DataFrame([[current_vehicle_df['path'].iloc[0],
                                          current_vehicle_df['id'].iloc[0],
                                          current_vehicle_df['day'].iloc[0],
                                          current_vehicle_df['time'].iloc[0],
                                          current_vehicle_df['smart_pole'].iloc[0],
                                          current_vehicle_df['category'].mode()[0],
                                          current_vehicle_df['time_slot'].iloc[0]]], columns=smaller_df.columns.values))
    #print(current_vehicle_df['category'].mode()[0])
    #print(smaller_df['id'].value_counts().idxmax())


In [ ]:
new_df.head(20)

In [ ]:
smaller_df.columns.values

In [ ]:
print(smaller_df['path'].values[:10])

In [ ]:
for image in X[1000:1010]:
    plt.imshow(image)
    plt.show()

In [ ]:
print(predictions)

In [ ]:
'15-29-59'

In [ ]:
time_slots = []
for current_time in df['time']:
    current_hour = int(current_time.split('-')[0])
    if current_hour == 1: 
        current_hour = int(current_time.split('-')[1])
    if current_hour == 7 or current_hour == 8:
        time_slots.append("7h-9h")
    elif current_hour == 9 or current_hour == 10:
        time_slots.append("9h-11h")
    elif current_hour == 15 or current_hour == 16:
        time_slots.append("9h-11h")
    else:
        print("pb here :", current_time)
        
        

In [ ]:
df['category'] = predictions

In [ ]:
df['time_slot'] = time_slots

In [ ]:
df.to_csv("inference_results.csv")

In [ ]:
print(confusion_matrix(np.argmax(y_test,axis=1), np.argmax(predictions,axis=1)))
print(accuracy_score(np.argmax(y_test,axis=1), np.argmax(predictions,axis=1)))

In [ ]:
predictions = model.predict(x_train)

In [ ]:
print(confusion_matrix(np.argmax(y_train,axis=1), np.argmax(predictions,axis=1)))
print(accuracy_score(np.argmax(y_train,axis=1), np.argmax(predictions,axis=1)))